# Introduction to Parallelization in GRASS GIS
The goal of parallelization is to speed up computation by using multiple cores. This notebooks introduces parallelization concepts, existing parallelized tools, and approaches to parallelizing user scripts.

<div class="alert alert-info">Throughout the workshop, we will be using shell syntax (Bash shell specifically) and Python. Some things are faster to write in Bash, and some in Python. By default, a cell in this notebook will interpret the code as Python, unless we use <code>%%bash</code> magic to let the notebook know a particular cell contains Bash syntax.</div>

First, let's download and unzip the [prepared dataset](https://doi.org/10.5281/zenodo.8206463) by executing the cell below:

In [ ]:
%%bash
sh download_dataset.sh

Let's start GRASS to run examples:

In [ ]:
import os
import subprocess
import sys

# Ask GRASS GIS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import GRASS packages
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
session = gj.init("opengeohub_2023/part_1");

The examples will run in *opengeohub_2023* project (previously called location) that contains sample data in the *PERMANENT* mapset (subproject). To keep things organized, this part of the workshop will run in a currently empty mapset *part_1*:

<img src="data_structure_embed.svg" alt="GRASS data structure" width="400"/>

### Measuring time
It's useful to be able to measure how much time a particular computation takes. We will be using `time` command in shell and `%%timeit` [IPython magic](https://ipython.readthedocs.io/en/stable/interactive/magics.html) for Python cells.

In [ ]:
%%bash
time sleep 1

Command time gives you 3 different numbers, but we are usually interested in the first one (real time), which is the real-life time it takes for a process to run from start to finish. The other two measure CPU time and you may see the "user" time can be larger than the "real" time for parallel tools.

In [ ]:
%%timeit  -n10 -r3
import time

def function(x):
    time.sleep(x)

function(0.1)

%%timeit magic will return elapsed time executing the Python cell. It typically runs the code multiple times to get a more accurate estimate. You can modify the number of loops (-n) and numer of repeated runs (-r).

##### *Try it yourself*
Using bash or Python, construct your own timed cell. You can use a sleep function or your own simple function (i.e. Hello World, 2+2). 

In [ ]:
# Try it here

### Running GRASS tools in Bash and Python
GRASS tools can be executed from Bash and Python (using the [grass.script package](https://grass.osgeo.org/grass-stable/manuals/libpython/script_intro.html) which is part of [GRASS GIS Python API](https://grass.osgeo.org/grass-stable/manuals/libpython/index.html). In the following example, you can print min and max values of a raster with [r.info](https://grass.osgeo.org/grass-stable/manuals/r.info.html) using the following Bash and Python syntax:

In [ ]:
%%bash
r.info map=DEM -r

In [ ]:
gs.run_command("r.info", map="DEM", flags="r")

## Using parallelized tools in GRASS GIS

There are many tools in GRASS GIS that are already parallelized ([see the list](https://grass.osgeo.org/grass-stable/manuals/keywords.html#parallel)). Many tools in GRASS Addons are parallelized as well.

Generally, there are two types of implementation in GRASS GIS.
Multithreading in C tools:
   * Threads have low overhead, so they can be spawned more efficiently.
   * Tools use OpenMP API. One of the advantages of OpenMP for software distribution is that code works (compiles and runs in serial) also without OpenMP library present on the system.
   * Memory is shared, so programmer needs to be cautious about race conditions (e.g., writing into the same variable).
   
Multiprocessing in Python tools:
   * There are multiple ways to implement it, typically tools use `subprocess` and `multiprocessing` package.
   * Python tools are often wrappers around GRASS tools implemented in C. For example, tool [r.sun.daily](https://grass.osgeo.org/grass-stable/manuals/addons/r.sun.daily.html) runs [r.sun](https://grass.osgeo.org/grass-stable/manuals/r.sun.html) for multiple days in parallel.
   
Parallelized tools have `nprocs` parameter to specify number of cores to use. For C tools using OpenMP, GRASS GIS needs to be compiled with OpenMP support to take advantage of it. Both implementations work well on a single machine, but can't be scaled to a distributed system. Scaling to a distributed system is covered at the end of this tutorial.


<div class="alert alert-info">The following examples assume an active GRASS session, meaning GRASS is started (or initialized in the notebook) in a specific project and mapset and tools are executed interactively. Towards the end of this notebook, we will cover running tools non-interactively (sometimes called batch mode).</div>

### Example
Let's run our first GRASS tool using multiple cores. Tool [r.neighbors](https://grass.osgeo.org/grass-stable/manuals/r.neighbors.html) computes moving window analysis, in this case we will smooth a digital elevation model. r.neighbors use OpenMP for parallelization.

In [ ]:
%%bash
time r.neighbors --q input=DEM output=DEM_smooth size=15 method=average nprocs=1
time r.neighbors --q input=DEM output=DEM_smooth size=15 method=average nprocs=2

The speedup (ratio of serial time to parallelized time with N cores) typically does not increase linearly with  the number of cores and parallel efficiency (speedup / N cores) decreases when adding cores. See, for example, the [benchmarks for r.neighbors](https://grass.osgeo.org/grass-stable/manuals/r.neighbors.html#performance). This behavior is due to the serial parts of the code (see [Amdahl's law](https://en.wikipedia.org/wiki/Amdahl%27s_law)) and computation overhead.

##### *Try it yourself*
What's the speedup for the r.neighbors command above with 2 cores? The efficiency?

In [ ]:
# Try it here

Let's use GRASS benchmarking library and matplotlib to look at this behavior in more detail. We will compare the time and parallel efficiency for different number of cores and different neighborhood window size of 3 and 9 cells.

In [ ]:
import grass.benchmark as bm
from grass.pygrass.modules import Module
from subprocess import DEVNULL

results = []
module = Module(
        "r.neighbors",
        input="DEM",
        output="benchmark",
        size=3,
        run_=False,
        stdout_=DEVNULL,
        overwrite=True,
    )
results.append(bm.benchmark_nprocs(module, label="size 3", max_nprocs=4, repeat=1))
module.inputs.size = 15
results.append(bm.benchmark_nprocs(module, label="size 15", max_nprocs=4, repeat=1))


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
plt.figure(figsize=(10,5))
plt.subplot(1, 3, 1)
plt.plot(results[0].nprocs, results[0].times, "-o", label="size_3")
plt.plot(results[1].nprocs, results[1].times, "-^", label="size_15")
plt.ylim(bottom=0)
plt.xlabel("Number of cores")
plt.ylabel("Time [s]")
plt.legend()
# plt.tight_layout()

plt.subplot(1, 3, 2)
plt.plot(results[0].nprocs, np.array(results[0].times[0]) / results[0].times, "-o", label="size_3")
plt.plot(results[1].nprocs, np.array(results[1].times[0]) / results[1].times, "-o", label="size_15")
# plt.plot(results[1].nprocs, results[1].efficiency, "-^", label="size_9")
# plt.ylim(bottom=0)
plt.xlabel("Number of cores")
plt.ylabel("Parallel speedup")
# plt.tight_layout()


plt.subplot(1, 3, 3)
plt.plot(results[0].nprocs, results[0].efficiency, "-o", label="size_3")
plt.plot(results[1].nprocs, results[1].efficiency, "-^", label="size_15")
plt.ylim(bottom=0)
plt.xlabel("Number of cores")
plt.ylabel("Parallel efficiency")
plt.tight_layout()

While computation with larger window size takes longer, it has better parallel efficiency, because a larger proportion of time is spent in the parallel part of the computation. As a rule of thumb, the parallel efficiency should be greater than 0.5 to not waste resources.

## Parallelization of workflows
In a geoprocessing workflow, there are often multiple independent tasks that can be executed in parallel.
The strategy how to divide the workflow into parallel tasks generally falls under either data-based or task-based parallelization.

Task-based parallelism partitions tasks so that independent tasks can be completed simultaneously.

With data-based parallelization, the spatial domain is partitioned for concurrent computations of individual spatial units 
and once processed, spatial units are mosaicked back into the initial spatial domain (if applicable).

<img src="parallelization_strategies.png" alt="data-based and task-based diagrams" width="600"/>

### Data-based parallelization
Data-based parallelism involves spatial domain decomposition, a process of splitting data into smaller datasets that can be processed in parallel.
As part of [GRASS GIS Python API](https://grass.osgeo.org/grass-stable/manuals/libpython/index.html), [GridModule](https://grass.osgeo.org/grass-stable/manuals/libpython/pygrass.modules.grid.html) decomposes input data into rectangular tiles, executes a given tool for each tile in parallel, and merges the results. Effectively, tiling is applied virtually (using computational region), determining the spatial extent of analysis for each parallel process. In some cases such as moving window analysis, tiles need to overlap to get correct results. Note that GridModule can be fairly inefficient due to the overhead from merging results back and is therefore best suited for computationally-itensive tasks such as interpolation.

The following example shows IDW interpolation split into 4 tiles. In this case, specifying an overlap is needed to get correct results without edge artifacts. Here, the number and size of tiles is automatically derived from the number of cores, but can be specified. First we create sample data for this example by taking 10,000 random sample points from the DEM.

In [ ]:
%%bash
g.region res=100
r.random -z input=DEM npoints=10000 vector=samples seed=1 --q

We also set the resolution for our IDW computation and sampling to 100 using the g.region tool.

<div class="alert alert-info">Each mapset is using a <strong>computational region</strong> that determines the extent and resolution of raster-based computations.
It can be changed with <a href="https://grass.osgeo.org/grass-stable/manuals/g.region.html">g.region</a> tool.</div>

In [ ]:
%%python
import time
from grass.pygrass.modules.grid import GridModule
import time
start = time.time()
grid = GridModule(
    "v.surf.idw",
    input="samples",
    output="idw",
    processes=4,
    overlap=20,
    quiet=True,
)
grid.run()
print(f"Elapsed time: {time.time() - start} s")

The following is the same tool ran in serial:

In [ ]:
%%bash
v.surf.idw --q input=samples output=idw

##### *Try it yourself*
Time the serial computation above. What is the speedup?

In [ ]:
# Try it here

There are tools that already integrate tiling. For example, addon [r.mapcalc.tiled](https://grass.osgeo.org/grass-stable/manuals/addons/r.mapcalc.tiled.html) uses the tiling concept for raster algebra computation. Using this is better for more complex algebra expression and large input data, otherwise the parallel efficiency of this method can be low.

In [ ]:
%%bash
g.region raster=DEM
r.mapcalc.tiled expression="vertical_sobel = -DEM[-1, -1] - 2 * DEM[0, -1] - DEM[1, -1] + DEM[-1, 1] + 2 * DEM[0, 1] + DEM[1, 1]" overlap=1 nprocs=4

### Task-based parallelization
With task-based parallelism, we identify independent tasks and execute them concurrently.
Tasks are typically GRASS processing tools executed as separate processes. Processes, unlike threads, do not share memory. When tasks are limited by disk I/O, parallel processing may have large overhead.


#### Examples in Python
There are multiple ways to execute tasks in parallel using Python, for example, there are libraries `multiprocessing` and `concurrent.futures`.

In the following example viewsheds from different coordinates are computed in parallel using `multiprocessing.Pool` class.

<div class="alert alert-warning">Note that Python multiprocessing.Pool examples do not work in an interactive interpreter (such as Jupyter Notebook). That's why we will first write a Python script with %%writefile and then use %run magic to execute it.</div>

In [ ]:
%%writefile example.py
import os
from multiprocessing import Pool
import grass.script as gs

def viewshed(point):
    x, y, cat = point
    gs.run_command("r.viewshed", input="DEM", output=f"viewshed_{cat}", coordinates=(x, y), maxdistance=10000)
    return f"viewshed_{cat}"

if __name__ == "__main__":
    viewpoints = [(594231, 275545, 1),
                  (659805, 259566, 2),
                  (646109, 232901, 3),
                  (602946, 203226, 4)]
    with Pool(processes=2) as pool:
        maps = pool.map(viewshed, viewpoints)
    print(maps)

In [ ]:
%%bash
g.region raster=DEM

In [ ]:
%run example.py

#### Examples in Bash
In the simplest case, tasks can be executed in parallel from a command line shell by running the geoprocessing tasks in the background (by appending `&`). Command `wait` forces to wait for the background processes to finish.

In [ ]:
%%bash
g.region raster=DEM res=100
v.kernel --q input=samples output=kernel_10000 radius=10000 &
v.kernel --q input=samples output=kernel_15000 radius=15000 &
wait

Larger number of tasks can be scheduled to run in parallel by tools such as [GNU Parallel](https://www.gnu.org/software/parallel/) and xargs.
In this simple example, we use a loop to write commands into a file and execute those commands in parallel, using 2 cores. 
Whenever a task is finished, a next one is picked from the queued tasks.


In [ ]:
%%bash
for R in 5000 10000 15000
do
    echo v.kernel --q input=samples output=kernel_${R} radius=${R} >> commands.sh
done
time parallel -j 2 < commands.sh

<div class="alert alert-info">While waiting, let's look at the processors working using <code>htop</code>. Click on the blue button New Launcher, create a terminal, and run htop.</div>

See manual pages of GNU Parallel or xargs for more advanced uses. GNU Parallel can be configured to distribute jobs across multiple machines. In that case, use `--exec` interface described below.

### Executing processes on distributed systems
To enable parallelization on distributed systems, tasks or scripts need to be executed non-interactively using the `--exec` [interface](https://grass.osgeo.org/grass-stable/manuals/grass.html) interface.
For that you need to specify Location and Mapset.


For example, here is a simple call to list all available rasters in PERMANENT mapset:


In [ ]:
%%bash
grass opengeohub_2023/PERMANENT --exec g.list type=raster mapset=PERMANENT

Non-interactive tasks need to be run in separate mapsets. One of the previous examples that was running within GRASS session in a single mapset can be rewritten so that each task runs in a newly created mapset. Note that by default newly created mapsets use default computational region for that GRASS location (you can use `g.region -s` to modify it). For raster computations, you need to change the computational region for each new mapset if the default one is not desired.

In [ ]:
%%bash
for R in 5000 10000 15000
do
    # first create a new mapset with -c flag and set computational region
    grass -c opengeohub_2023/mapset_${R} --exec g.region raster=DEM res=100
    # write the command executing v.kernel in the newly created mapset to a file
    echo grass opengeohub_2023/mapset_${R} --exec v.kernel --q input=samples@part_1 output=kernel_${R} radius=${R} >> exec_commands.sh
done
parallel -j 2 < exec_commands.sh

In some cases, only a temporary mapset or location is needed, see [examples](https://grass.osgeo.org/grass-stable/manuals/grass.html#batch-jobs-with-the-exec-interface).
Besides individual tools, the `--exec` interface can execute an entire script to enable more complex workflows.

## Safe execution of parallel tasks (advanced)

There are certain situations you need to avoid when running GRASS tools in parallel:

 * write output maps/files with identical names
 * modify computational region when running within the same mapset
 * modify MASK when running within the same mapset
 * modify vector attribute database within the same mapset
 * use [r.reclass](https://grass.osgeo.org/grass-stable/manuals/r.reclass.html) to reclassify from the same base map


The following examples show how to deal with some of these situations when running parallel tasks in the same mapset is desired.

### Safely modifying computational region in a single mapset

Sometimes modifying computational region in a script is needed. It is a good practice to not change the global computational region, which effectively modifies a file in a mapset,
but only change the environment variable `GRASS_REGION`.
Here, we modified the previous viewshed example to compute in parallel viewsheds with different extents:

In [ ]:
%%writefile example.py
import os
from multiprocessing import Pool
import grass.script as gs

def viewshed(point):
    x, y, cat = point
    # copy current environment, modify and pass it to r.viewshed
    env = os.environ.copy()
    maxdistance = 10000
    # create region based on the viewpoint and maxdistance
    env["GRASS_REGION"] = gs.region_env(e=x + maxdistance, w=x - maxdistance, n=y + maxdistance, s=y - maxdistance, align="DEM")
    gs.run_command("r.viewshed", input="DEM", output=f"viewshed_{cat}", coordinates=(x, y), max_distance=maxdistance, env=env)
    return f"viewshed_{cat}"

if __name__ == "__main__":
    viewpoints = [(594231, 275545, 1),
                  (659805, 259566, 2),
                  (646109, 232901, 3),
                  (602946, 203226, 4)]
    with Pool(processes=2) as pool:
        maps = pool.map(viewshed, viewpoints)
    print(maps)

In [ ]:
%run example.py

### Safely modifying vectors with attributes in a single mapset

By default vector maps share a single SQLite database file, however SQLite does not support concurrent write access. That poses a problem when modifying vectors with attributes in parallel. While this can be solved by running the computations in separate mapsets, it is also possible to change the default behavior to write attributes of each vector to the vector's individual SQLite file. This behavior can be activated after a new mapset is created with:

```
 db.connect driver=sqlite database='$GISDBASE/$LOCATION_NAME/$MAPSET/vector/$MAP/sqlite.db'
```

Alternatively, a PostgreSQL or another database backend can be used for attributes to offload the parallel writing to the database system.

## Exercise

Consider this (slightly simplified) example code taken from the [r.sun documentation](https://grass.osgeo.org/grass-stable/manuals/r.sun.html):
```
# slope + aspect
r.slope.aspect elevation=DEM aspect=aspect slope=slope

# calculate global irradiation for day 180
r.sun elevation=DEM aspect=aspect slope=slope glob_rad=global_rad day=180 nprocs=2
# result: output global (total) irradiation [Wh.m-2.day-1] for given day
r.univar global_rad
```

Use [r.sun](https://grass.osgeo.org/grass-stable/manuals/r.sun.html) to compute the solar radiation on the winter and summer soltices (day of year 355 and 172 respectively), the spring equinox (day 80) and your birthday (you can look it up [here](https://asd.gsfc.nasa.gov/Craig.Markwardt/doy2023.html)). Try to parallelize the workflow while considering available resources. What strategies for parallelizing have you used? 

In [ ]:
# Your work here (use Bash or Python)